<a href="https://colab.research.google.com/github/com0040/offline_alpaco/blob/main/%EC%A0%84%EC%B2%B4_%EC%9B%B9%ED%88%B0_%EC%B5%9C%EC%8B%A0%ED%99%94_%EB%B2%A0%EC%8A%A4%ED%8A%B8_%EB%8C%93%EA%B8%80(%EC%8B%A4%EC%8B%9C%EA%B0%84_%EC%BD%94%EB%94%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 검색한 웹툰 크롤링
from IPython.utils.strdispatch import re
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import json

all_web_url = 'https://comic.naver.com/webtoon/weekday'
resp0 = requests.get(all_web_url)
soup0 = BeautifulSoup(resp0.content, 'lxml')
latest_tags = soup0.select('div.list_area.daily_all div.thumb a')
a_tags = soup0.select('div.list_area.daily_all li a.title')

tits, urls,ids, no = [],[],[],[]

for a_tag in a_tags:
    tits.append(a_tag['title'])
    urls.append('https://comic.naver.com' + a_tag['href'])
    
    id = a_tag['href'].split('titleId=')[1].split('&')[0]
    # print(id)
    # break
    ids.append(id)

for url in urls:
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    a_tags = soup.select('table.viewList td.title a')
    no.append(int(a_tags[0]['href'].split('&')[1].replace('no=', '')))




In [ ]:
toon_info = {}

for i in range(len(tits)):
    toon_info[tits[i]] = {'title_id' : ids[i], 'no' : int(no[i])}

toon_info

{'참교육': {'title_id': '758037', 'no': 103},
 '뷰티풀 군바리': {'title_id': '648419', 'no': 358},
 '퀘스트지상주의': {'title_id': '783052', 'no': 54},
 '장씨세가 호위무사': {'title_id': '728750', 'no': 181},
 '윈드브레이커': {'title_id': '602910', 'no': 412},
 '퍼니게임': {'title_id': '801035', 'no': 5},
 '팔이피플': {'title_id': '774863', 'no': 55},
 '신화급 귀속 아이템을 손에 넣었다': {'title_id': '795297', 'no': 23},
 '버림받은 왕녀의 은밀한 침실': {'title_id': '796867', 'no': 18},
 '호랑신랑뎐': {'title_id': '795643', 'no': 43},
 '앵무살수': {'title_id': '739115', 'no': 127},
 '소녀의 세계': {'title_id': '654774', 'no': 340},
 '잔불의 기사': {'title_id': '768536', 'no': 87},
 '백수세끼': {'title_id': '733074', 'no': 147},
 '똑 닮은 딸': {'title_id': '774866', 'no': 60},
 '사이다걸': {'title_id': '800333', 'no': 7},
 '절대검감': {'title_id': '796075', 'no': 20},
 '히어로메이커': {'title_id': '21815', 'no': 846},
 '물어보는 사이': {'title_id': '772725', 'no': 65},
 '리턴 투 플레이어': {'title_id': '752414', 'no': 100},
 '꼬리잡기': {'title_id': '758666', 'no': 67},
 '인섹터': {'title_id': '801710', 'no': 

In [ ]:
# 2. 원하는 웹툰 제목 입력 -> 해당 웹툰의 고유아이디, 최신화정보 조회
mytoon = input('어떤 웹툰의 댓글을 크롤링해올까요?')

어떤 웹툰의 댓글을 크롤링해올까요?호랑이형님


In [ ]:
print(toon_info[mytoon]['no'])
print(toon_info[mytoon]['title_id'])

my_new_no = toon_info[mytoon]['no']
my_title_id = toon_info[mytoon]['title_id']

357
650305


### 웹툰 댓글 크롤링 마지막 단계
함수화 작업
- 입력인자 = (웹툰 제목, 베스트/일반 설정)
- 출력 = 해당 웹툰의 전체화 댓글, 공감, 비공감, 화 + 추가(선택)

In [ ]:
# 3. 해당 웹툰의 전체화 베스트댓글 크롤링
## 댓글 크롤링 모듈 만들기 (입력파라미터 : 웹툰고유아이디, 최신화정보, 베스트댓글(디폴트)/전체댓글)
##                         (return 값 : reviews)

def toon_review_scraping(last_no, title_id):

    title_id = toon_info[mytoon]['title_id']
    no = toon_info[mytoon]['no']

    reviews = {
        '댓글' : [],
        '공감수' : [],
        '비공감수' : [],
        '화' :[]
    }

    for num in range(1,no+1):

        url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'


        params ={
                'ticket' : 'comic',  
                'templateId' : 'webtoon',  
                'pool' : 'cbox3',  
                '_callback' : 'jQuery11240590927015056502_1658103118387', # jQuery112409091552467251651_1615854232166 -> ''  
                'lang' : 'ko',  
                'country' : 'KR',  
                'objectId' : '795487_26', # 웹툰의 고유 아이디_해당화 인덱스  
                'pageSize' : 15,  
                'indexSize' : 10,  
                'listType' : 'OBJECT',  
                'pageType' : 'default',  
                'page' : 1, # 전체댓글의 페이지 번호 
                'refresh' : 'true',  
                }

        headers = {
                'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
        }

        resp = requests.get(url, params=params, headers=headers)
        
        resp

        jsn = resp.text[41:-2]

        dict_resp = json.loads(jsn)
        dict_resp

        commentList = dict_resp['result']['commentList']
        # len(commentList)

        for comment in commentList:
            reviews['댓글'].append(comment['contents'])
            reviews['공감수'].append(comment['sympathyCount'])
            reviews['비공감수'].append(comment['antipathyCount'])
            reviews['화'].append(no)

    return reviews


In [ ]:
toon_info

{'참교육': {'title_id': '758037', 'no': 103},
 '뷰티풀 군바리': {'title_id': '648419', 'no': 358},
 '퀘스트지상주의': {'title_id': '783052', 'no': 54},
 '장씨세가 호위무사': {'title_id': '728750', 'no': 181},
 '윈드브레이커': {'title_id': '602910', 'no': 412},
 '퍼니게임': {'title_id': '801035', 'no': 5},
 '팔이피플': {'title_id': '774863', 'no': 55},
 '신화급 귀속 아이템을 손에 넣었다': {'title_id': '795297', 'no': 23},
 '버림받은 왕녀의 은밀한 침실': {'title_id': '796867', 'no': 18},
 '호랑신랑뎐': {'title_id': '795643', 'no': 43},
 '앵무살수': {'title_id': '739115', 'no': 127},
 '소녀의 세계': {'title_id': '654774', 'no': 340},
 '잔불의 기사': {'title_id': '768536', 'no': 87},
 '백수세끼': {'title_id': '733074', 'no': 147},
 '똑 닮은 딸': {'title_id': '774866', 'no': 60},
 '사이다걸': {'title_id': '800333', 'no': 7},
 '절대검감': {'title_id': '796075', 'no': 20},
 '히어로메이커': {'title_id': '21815', 'no': 846},
 '물어보는 사이': {'title_id': '772725', 'no': 65},
 '리턴 투 플레이어': {'title_id': '752414', 'no': 100},
 '꼬리잡기': {'title_id': '758666', 'no': 67},
 '인섹터': {'title_id': '801710', 'no': 

In [ ]:
reviews = {
        '댓글' : [],
        '공감수' : [],
        '비공감수' : [],
        '제목' :[]
    }

for title, info in toon_info.item:
    title_id = info['title_id']
    no = info['no']


    for num in range(1,no+1):

        url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'


        params ={
                'ticket' : 'comic',  
                'templateId' : 'webtoon',  
                'pool' : 'cbox3',  
                '_callback' : '', # jQuery112409091552467251651_1615854232166 -> ''  
                'lang' : 'ko',  
                'country' : 'KR',  
                'objectId' : f'{title_id}_{no}', # 웹툰의 고유 아이디_해당화 인덱스  
                'pageSize' : 15,  
                'indexSize' : 10,  
                'listType' : 'OBJECT',  
                'pageType' : 'default',  
                'page' : 1, # 전체댓글의 페이지 번호 
                'refresh' : 'true',  
                }

        headers = {
                'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
        }

        resp = requests.get(url, params=params, headers=headers)
        
        resp

        jsn = resp.text[10:-2]

        dict_resp = json.loads(jsn)
        dict_resp

        commentList = dict_resp['result']['commentList']
        # len(commentList)

        for comment in commentList:
            reviews['댓글'].append(comment['contents'])
            reviews['공감수'].append(comment['sympathyCount'])
            reviews['비공감수'].append(comment['antipathyCount'])
            reviews['제목'].append(title)

In [ ]:
# 내가 짠거

params ={ # 해야할 이유가 있을 때만
            'ticket' : 'comic',  
            'templateId' : 'webtoon',  
            'cv' : '20221018144728',
            'pool' : 'cbox3',  
            '_callback' : '', # _callback=Query112401837211841906185j6_16672011281816 -> ''  
            'lang' : 'ko',  
            'country' : 'KR',  
            'objectId' : '741449_1', # 웹툰의 고유 아이디_해당화 인덱스  # 전체 웹툰의 고유아이디를 for문
            'pageSize' : 15,  
            'indexSize' : 10,  
            'listType' : 'OBJECT',  
            'pageType' : 'default',  
            'page' : 1, # 전체댓글의 페이지 번호 
            'refresh' : 'true',  
            'replyPageSize':'10',
            'sort': 'NEW',
            'initialize' : 'true',
            'useAltSort':'true',
            '_': '1667201128183'
            }

headers ={
            'referer' : 'https://comic.naver.com/comment/comment?titleId=801505&no=1', # 누구 소개?
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    }

web_name_list, comment_list, sympathy, antipathy  = [],[],[],[]

for idx, latest_tag in enumerate(latest_tags): # 모든 웹툰
    web_id = latest_tag['href'].split('titleId=')[1].split('&')[0]
    
    thumb_url = 'https://comic.naver.com' + latest_tag['href']    
    resp1 = requests.get(thumb_url)    
    soup1 = BeautifulSoup(resp1.content, 'lxml')

    latest_num_tags = soup1.select('table.viewList td.title a')
    latest_num = int(latest_num_tags[0]['href'].split('no=')[1].split('&')[0])
    # print(type(latest_num))
    for i in range(1, latest_num + 1): # 최신화까지 반복
        
        params['objectId'] = str(web_id) + '_{}'.format(i)
        comment_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'
        resp2 = requests.get(comment_url, params = params, headers = headers)
        # print(resp.text)
        jsn1 = resp2.text[10:-2]
        dic_comment1 = json.loads(jsn1)
        if i ==1:
            page_num = int(dic_comment1['result']['pageModel']['totalPages'])
        
        for j in range(1, page_num + 1): # 페이지수만큼 반복
            params['page'] = '{}'.format(j)
            resp3 = requests.get(comment_url, params =params, headers = headers)
            jsn2 = resp3.text[10:-2]
            dic_comment2 = json.loads(jsn2)

            comments = dic_comment2['result']['commentList']
            for comment in comments: # 코멘트 담기
                comment_list.append(comment['contents'])
                sympathy.append(comment['sympathyCount']) # 공감 
                antipathy.append(comment['antipathyCount']) # 비공감 
                web_name_list.append(tits(i))
                break
            break
        break
    
    break


### post로 된 경우 <- get 대신

- 네트워크 탭 : headers -> request URL 아래

request.post(url)로 요청해야함(*4가지 방법 : 클라 <-> 섭 소통)

클라이언트 요청 mathod

1. post : create - 

2. get : read

3. update : update

4. delete : delete


